In [ ]:
# code/mbatch_i-o.ipynb

#### Split and merge by sys for mbatch

In [ ]:
%whos

In [ ]:
import pandas as pd
import glob

In [ ]:
ls mbatch/bk_mbatch/data/

In [ ]:
mkdir mbatch/bk_mbatch/data/r5_splt_in/

In [ ]:
mkdir mbatch/bk_mbatch/data/mbatch_r5/

In [ ]:
rm -r mbatch/bk_mbatch/data/r5/

In [ ]:
ls mbatch/bk_mbatch/data/

In [ ]:
# Split files from ../results/input-data/r5/r5-fls/
    # write split files to r5_splt_in/
    # command line from docker container on that
    # send to mkdir mbatch/bk_mbatch/data/mbatch_r5/

    # checking for negs on front end, pull 7 completed files from r4 to r5
"""
     * Paths are tied to Docker loc *
"""

In [ ]:
r = 'r5'

In [ ]:
ls ../results/input-data/r5/r5-fls/

In [ ]:
r5_pths = glob.glob('../results/input-data/r5/r5-fls/*.tsv')

In [ ]:
ls ../results/mbatch-out/r4/r4-fls/

In [ ]:
r5_pths

In [ ]:
# Does it make biological sense to correct between cell line and HCMI?

In [ ]:
systems.split('+')[0]

##### NMF zeros

In [ ]:
# Duplicate all genes, so apply to columns 20K to 40K
P1: 4,6,-3,4,-1,4 becomes two rows
P1_pos: 4,6,0,4,0,4
P2_neg: 0,0,3,0,1,0

#### r5 production run

In [ ]:
r = 'r5'
value_counts = []
for fl_pth in r5_pths:
    print(fl_pth.split('/')[-1])
    ab = pd.read_csv(
        fl_pth,
        sep = '\t', index_col = 0)
    print('min val ', ab.iloc[:, 2:].min().min())
    if ab.iloc[:, 2:].min().min() < 0:
        print('neg values')
        print(' ')
        continue # to next
    print('pos vals, spitting to disk', r)
    print(' ')
    # break
    cancer = fl_pth.split('_')[0].split('/')[-1]
    modality = fl_pth.split('_')[1].split('/')[-1]
    systems = fl_pth.split('_')[-1].split('.')[0]
    # break
    src_ref = 'src'
    sys = systems.split('+')[0] # Automation note: human always last
    cl = ab[ab.System == sys].copy()
    cl.drop(columns = ['System'], inplace = True)
    cl.drop(columns = ['Cancer_type'], inplace = True)
    cl = cl.T
    # break
    cl.to_csv(
              'mbatch/bk_mbatch/data/'+r+'_splt_in/'+
              cancer+'_'+sys+'_'+modality+'_'+src_ref+'.tsv',
              sep = '\t')

    src_ref = 'ref'
    sys = systems.split('+')[1]
    cp = ab[ab.System == 'CPTAC'].copy()    # Label datum to file name
    cp.drop(columns = ['System'], inplace = True)
    cp.drop(columns = ['Cancer_type'], inplace = True)
    cp = cp.T
    cp.to_csv(
              'mbatch/bk_mbatch/data/'+r+'_splt_in/'+
              cancer+'_'+sys+'_'+modality+'_'+src_ref+'.tsv',
              sep = '\t')

##### r4 (template)

In [ ]:
one_cncr_files = [
    '../strctrd/one_cncr/breast-lobular_transcriptomics_cell-line+CPTAC.tsv',
      # '../strctrd/one_cncr/beat-aml_transcriptomics_cell-line+CPTAC.tsv'
    ]

In [ ]:
run = 'r4'
value_counts = []
for file in one_cncr_files:
    print(file)
    ab = pd.read_csv(
        file,
        sep = '\t', index_col = 0)
    print('min val ', ab.iloc[:, 2:].min().min())
    value_counts.append([ab.System.value_counts(),
                         ab.Cancer_type.value_counts()])
    cancer = file.split('_')[1].split('/')[1]
    modality = file.split('_')[2]

    src_ref = 'src'
    sys = 'cell_line'
    cl = ab[ab.System == sys].copy()
    cl.drop(columns = ['System'], inplace = True)
    cl.drop(columns = ['Cancer_type'], inplace = True)
    cl = cl.T
    # break
    cl.to_csv(
              'mbatch/bk_mbatch/data/'+run+'_splt_in/'+
              cancer+'_'+sys+'_'+modality+'_'+src_ref+'.tsv',
              sep = '\t')
    # ^glob this dir, to mbatch_one_cncr.R

    src_ref = 'ref'
    sys = 'cptac'
    cp = ab[ab.System == 'CPTAC'].copy()    # Label datum to file name
    cp.drop(columns = ['System'], inplace = True)
    cp.drop(columns = ['Cancer_type'], inplace = True)
    cp = cp.T
    cp.to_csv(
              'mbatch/bk_mbatch/data/'+run+'_splt_in/'+
              cancer+'_'+sys+'_'+modality+'_'+src_ref+'.tsv',
              sep = '\t')
    # break

In [ ]:
sys, cancer, modality, src_ref

##### r4, fix brca

In [ ]:
'mbatch/bk_mbatch/data/r4_splt_in

beat aml trans was neg

##### run 1

In [ ]:
# Target output dir for one cncr split files:
bk_mbatch/data/oc_splt_in
# (Docker command line R script output loc)

#### Mbatch out to results/mbatch-out/rn

In [ ]:
# already in r5 from r4

In [ ]:
ls ../results/mbatch-out/r5/r5-fls

In [ ]:
mb_n = sorted(glob.glob(
    'mbatch/bk_mbatch/data/mbatch_'+r+'/correctedResults_src_r*.tsv'))

In [ ]:
mb_n

In [ ]:
# From R script 5
cancer_type = [
    'pancreatic',
    'ovarian',
    'lung-sq',
    'lung-ad',
    'breast',
    'brain',
    ]

In [ ]:
moda_type = [
    # 'pancreatic',
    # 'ovarian',
    # 'lung-sq',
    # 'lung-ad',
    # 'breast',
    # 'brain',
    ]

In [ ]:
sys_type = [
    'HCMI+CPTAC',
    'HCMI+CPTAC',
    'lung-sq',
    'lung-ad',
    'breast',
    'brain',
    ]

In [ ]:
# r5 rebuild
for i, strctrd_dot_T_file in enumerate(mb_n):
    print(strctrd_dot_T_file)
    strctrd_dot_T = pd.read_csv(
    strctrd_dot_T_file,
        sep = '\t', index_col = 0)

    # strctrd_dot_T
    s = strctrd_dot_T.T

    s.insert(0, 'Cancer_type', cancer_type[i])

    src_df = s[s.index.str.endswith(".src")]
    ref_df = s[s.index.str.endswith(".ref")]

    src_df.insert(0, 'System', 'cell_line')
    ref_df.insert(0, 'System', 'CPTAC')

    mb_strctrd = pd.concat([src_df, ref_df], axis = 0)

    cancer = cancer_type[i]
    modality = 'transcriptomics'
    # systems = 'cell-line+CPTAC'
    # systems = sys_type[i]
    systems = 'HCMI+CPTAC'
    mb_strctrd.to_csv(
        '../results/mbatch-out/'+r+'/'+r+'-fls/'+cancer+'_'+modality+'_'+systems+'.tsv',
        sep = '\t')
    # break

In [ ]:
# end v5

#### Mbatch out .T to strctrd (template)

##### run one, og nine

In [ ]:
ls mbatch/bk_mbatch/data/mbatch_oc/

##### r4

In [ ]:
ls mbatch/bk_mbatch/data/r4_splt_in/

In [ ]:
run # was 'oc' before r4 (one cancer, nine files)

In [ ]:
ls mbatch/bk_mbatch/data/mbatch_r4

In [ ]:
mb_n = sorted(glob.glob(
    'mbatch/bk_mbatch/data/mbatch_'+run+'/correctedResults_src_r*.tsv'))

In [ ]:
mb_n

In [ ]:
# mbatch_r4.R
# breast-lobular_transcriptomics_cell-line+CPTAC

In [ ]:
cancer_type = [#'breast-ductal',
'breast-lobular',
# 'breast-nos',
# 'colon-adeno',
# 'glioblastoma',
# 'head-neck',
# 'lung-adeno',
# 'pancreatic-adeno',
# 'renal-clear-cell'
]

In [ ]:
# r4 rebuild, used as template for v5
for i, strctrd_dot_T_file in enumerate(mb_n):
    print(strctrd_dot_T_file)
    strctrd_dot_T = pd.read_csv(
    strctrd_dot_T_file,
        sep = '\t', index_col = 0)

    strctrd_dot_T
    s = strctrd_dot_T.T

    s.insert(0, 'Cancer_type', cancer_type[i])

    src_df = s[s.index.str.endswith(".src")]
    ref_df = s[s.index.str.endswith(".ref")]

    src_df.insert(0, 'System', 'cell_line')
    ref_df.insert(0, 'System', 'CPTAC')

    mb_strctrd = pd.concat([src_df, ref_df], axis = 0)

    cancer = cancer_type[i]
    modality = 'transcriptomics'
    systems = 'cell-line+CPTAC'
    mb_strctrd.to_csv(
        '../results/mbatch_out/'+run+'/files/'+cancer+'_'+modality+'_'+systems+'.tsv',
        sep = '\t')
    # break

In [ ]:
lst_file_out = '../results/mbatch/one_cncr/files/'+cancer+'_'+modality+'_'+systems+'.tsv'

In [ ]:
chk = pd.read_csv(
    lst_file_out,
        sep = '\t', index_col = 0)

In [ ]:
chk

##### devel notes on one cancer return to strctrd frmt:

In [ ]:
for indx in strctrd.index:
    print(indx) # observation: mbatch output is sorted alphanum

In [ ]:
ab = glob.glob('../strctrd/one_cncr/'+cancer_type+'*')

#### Two-cancer version

##### read and check

In [ ]:
ab = pd.read_csv( # changed to strctrd/one_cncr above
    '../output/dbl-cncr/pancreatic-adeno+lung-adeno_transcriptomics_cell-line+CPTAC.tsv',
        sep = '\t', index_col = 0)

In [ ]:
# mbatch takes non-negative numbers
ab.iloc[:, 2:].min().min()

In [ ]:
ab.System.value_counts()

In [ ]:
ab.Cancer_type.value_counts()

In [ ]:
cancers = 'pancreatic-adeno+lung-adeno'

In [ ]:
modality = 'transcriptomics'

##### Split template, using dep output/ paths

In [ ]:
src_ref = 'src'
sys = 'cell_line'
cl = ab[ab.System == 'cell_line'].copy()
cl.drop(columns = ['System'], inplace = True)
cl.drop(columns = ['Cancer_type'], inplace = True)
cl = cl.T
cl.to_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
    sep = '\t')

In [ ]:
cl_transposed_read_test = pd.read_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
        sep = '\t',
        index_col = 0)

In [ ]:
cl_transposed_read_test

In [ ]:
src_ref = 'ref'
sys = 'cptac'
cp = ab[ab.System == 'CPTAC'].copy()
cp.drop(columns = ['System'], inplace = True)
cp.drop(columns = ['Cancer_type'], inplace = True)
cp = cp.T
cp.to_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
    sep = '\t')

In [ ]:
cp_transposed_read_test = pd.read_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
        sep = '\t',
        index_col = 0)
cp_transposed_read_test

In [ ]:
ab

In [ ]:
ab.System.value_counts()

#### Attach 2 label sets

In [ ]:
ls # this is mbatch_i-o.ipynb

In [ ]:
ls mbatch/bk_mbatch/data/mbatch/

In [ ]:
cd = pd.read_csv(
    'mbatch/bk_mbatch/data/mbatch/correctedResults_src_v1.tsv',
        sep = '\t', index_col = 0)

In [ ]:
cd

In [ ]:
src_df = cd.T[cd.T.index.str.endswith(".src")]
ref_df = cd.T[cd.T.index.str.endswith(".ref")]

In [ ]:
src_df

In [ ]:
ref_df

In [ ]:
ab_src = ab[ab.System == 'cell_line']
ab_ref = ab[ab.System == 'CPTAC']

In [ ]:
ab_src

In [ ]:
ab_ref

In [ ]:
src_df.insert(0, 'Cancer_type', list(ab_src.Cancer_type))
ref_df.insert(0, 'Cancer_type', list(ab_ref.Cancer_type))

In [ ]:
src_df.insert(0, 'System', 'cell_line')
ref_df.insert(0, 'System', 'CPTAC')

In [ ]:
mb_sys = pd.concat([src_df, ref_df], axis = 0)

In [ ]:
mb_sys

#### Run random forest from front-end on mb_sys

In [ ]:
systems = 'cell-line+CPTAC'

In [ ]:
cancer, modality, systems

In [ ]:
mb_sys.to_csv(
    '../results/mbatch/files/'+cancer+'_'+modality+'_'+systems+'.tsv',
    sep = '\t')

In [ ]:
obj_n = pd.read_csv(
    '../results/mbatch/files/'+cancer+'_'+modality+'_'+systems+'.tsv',
    sep = '\t', index_col = 0)

In [ ]:
del(mb_sys)

In [ ]:
import pandas as pd
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import statistics

In [ ]:
obj_n

##### Random forest, for delete

In [ ]:
X = obj_n.iloc[:, 2:]
y_sys = obj_n.System
y_c_typ = obj_n.Cancer_type

sys_scores = []
c_typ_scores = []

for i in list(range(0,9)):
    X_train, X_test, y_train, y_test = train_test_split(X, y_sys)
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average = 'weighted')
    sys_scores.append(score)

    y_train = y_c_typ[y_c_typ.index.isin(y_train.index)]
    y_test = y_c_typ[y_c_typ.index.isin(y_test.index)]
    
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average = 'weighted')
    c_typ_scores.append(score)

In [ ]:
cancers+'_'+modality+'_'+systems

In [ ]:
file_n = cancers+'_'+modality+'_'+systems

In [ ]:
sys_mean = statistics.mean(sys_scores)
c_typ_mean = statistics.mean(c_typ_scores)

sys_err = statistics.stdev(sys_scores)
c_typ_err = statistics.stdev(c_typ_scores)

fig, ax = plt.subplots()

# Create bars
bar1 = ax.bar('sys', sys_mean, yerr=sys_err, capsize = 7, label='System')
bar2 = ax.bar('c typ', c_typ_mean, yerr=c_typ_err, capsize = 7, label='Cancer type')

plt.title(file_n) # automate later
plt.xlabel('Cancer Model System')
plt.ylabel('F1 score')

# Legend
ax.legend()
plt.savefig('../results/mbatch/figures/'+file_n+'rf_barplot.png')